In [1]:
# 1. Încarcă arhiva
from google.colab import files
uploaded = files.upload()


Saving hopular-main.zip to hopular-main (1).zip


In [2]:
# 2. Dezarhivează
!unzip hopular-main.zip
%cd hopular-main

# 3. Instalează Hopular local
!pip install -e .


Archive:  hopular-main.zip
3e0c39fdc59568349373af573ee52c03305ca105
replace hopular-main/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
/content/hopular-main
Obtaining file:///content/hopular-main
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/ml-jku/hopfield-layers to /tmp/pip-install-x_dk8swl/hopfield-layers_40584a34b0a04c318e410af0b40b4a0e
  Running command git clone --filter=blob:none --quiet https://github.com/ml-jku/hopfield-layers /tmp/pip-install-x_dk8swl/hopfield-layers_40584a34b0a04c318e410af0b40b4a0e
  Resolved https://github.com/ml-jku/hopfield-layers to commit f56f929c95b77a070ae675ea4f56b6d54d36e730
  Preparing metadata (setup.py) ... done
  Attempting uninstall: hopular
    Found existing installation: hopular 1.0.0
    Uninstalling hopular-1.0.0:
      Successfully uninstalled hopular-1.0.0
  Running setup.py develop for hopular


In [3]:
!pip install fairscale pytorch-lightning --quiet



First, we import the necessary libraries, including PyTorch for building the neural network, scikit-learn for data handling, and the specific modules from `hopular` and `hflayers`.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from hopular.blocks import EmbeddingBlock
from hflayers import Hopfield
import math

Next, we load the Breast Cancer dataset from scikit-learn, preprocess the data by scaling it, split it into training and testing sets, and convert the data into PyTorch tensors.

In [5]:
# Încărcare set de date
data = load_breast_cancer()
X, y = data.data, data.target

# Normalizare
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Împărțire în train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conversie în tensori PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

We then define the model parameters, including the input dimension, feature size for embeddings, dropout rate, and specify that all features are continuous.

In [6]:
input_dim = X.shape[1]
feature_size = 64
dropout = 0.1
discrete_features = torch.tensor([], dtype=torch.long)  # toate sunt continue

This cell defines the `HopularModel` class, which is a PyTorch neural network. It includes an `EmbeddingBlock` to process the input features, a `Hopfield` layer applied individually to each feature's embedding, and a final linear layer for classification. The `forward` method outlines how data flows through the network.

In [7]:
class HopularModel(nn.Module):
    def __init__(self, input_dim, feature_size, dropout, discrete_features, num_classes):
        super().__init__()
        self.embed = EmbeddingBlock(
            input_sizes=[1] * input_dim,
            feature_size=feature_size,
            feature_discrete=discrete_features,
            dropout_probability=dropout
        )
        self.hopfield = Hopfield(
            input_size=feature_size,
            hidden_size=feature_size,
            output_size=feature_size,
            pattern_size=feature_size,
            num_heads=1,
            scaling=1.0 / math.sqrt(feature_size),
            dropout=dropout,
            update_steps_max=1
        )
        self.output_layer = nn.Linear(input_dim * feature_size, num_classes)
        self.input_dim = input_dim
        self.feature_size = feature_size

    def forward(self, x):
        emb = self.embed(x)  # Assuming this outputs [B, F*E]

        # Reshape emb to [B, F, E]
        emb = emb.view(-1, self.input_dim, self.feature_size)

        # Apply Hopfield to each feature's embedding individually
        attended_features = []
        for i in range(emb.shape[1]):
            feature_embedding = emb[:, i, :] # [B, E]
            # Unsqueeze to add a pattern dimension [B, 1, E]
            feature_embedding = feature_embedding.unsqueeze(1)
            # Pass to Hopfield layer
            attended_feature = self.hopfield(feature_embedding) # [B, 1, E]
            # Squeeze back to [B, E]
            attended_features.append(attended_feature.squeeze(1))

        # Concatenate attended features back along the feature dimension
        attended = torch.stack(attended_features, dim=1) # [B, F, E]

        flat = attended.view(attended.shape[0], -1)  # [B, F, E] -> [B, F*E]
        return self.output_layer(flat)

# Instantiate the new model
model = HopularModel(input_dim, feature_size, dropout, discrete_features, num_classes=2)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

This cell contains the training loop. The model is trained for 20 epochs using the Adam optimizer and Cross-Entropy Loss. The loss is printed after each epoch to monitor the training progress.

In [8]:
# Antrenare 20 epoci
for epoch in range(20):
    model.train() # Set the model to training mode
    model_out = model(X_train_tensor)
    loss = loss_fn(model_out, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")

Epoch 1 Loss: 0.6910
Epoch 2 Loss: 0.6566
Epoch 3 Loss: 0.6300
Epoch 4 Loss: 0.6078
Epoch 5 Loss: 0.5875
Epoch 6 Loss: 0.5603
Epoch 7 Loss: 0.5324
Epoch 8 Loss: 0.4957
Epoch 9 Loss: 0.4524
Epoch 10 Loss: 0.4071
Epoch 11 Loss: 0.3616
Epoch 12 Loss: 0.3129
Epoch 13 Loss: 0.2658
Epoch 14 Loss: 0.2279
Epoch 15 Loss: 0.1948
Epoch 16 Loss: 0.1660
Epoch 17 Loss: 0.1500
Epoch 18 Loss: 0.1395
Epoch 19 Loss: 0.1235
Epoch 20 Loss: 0.1249


Finally, we evaluate the trained model on the test set to measure its performance. The accuracy is calculated and printed.

In [9]:
model.eval()
with torch.no_grad():
    model_out = model(X_test_tensor)
    predictions = torch.argmax(model_out, dim=1)
    acc = (predictions == y_test_tensor).float().mean().item()
    print(f"\nTest accuracy: {acc * 100:.2f}%")


Test accuracy: 99.12%
